In [2]:
import psycopg2
from psycopg2.extras import execute_batch
from faker import Faker
import random
from datetime import datetime, timedelta
import os
import configparser

fake = Faker()
Faker.seed(0)


# Determine the base directory
try:
    base_dir = os.path.dirname(os.path.abspath(__file__))  # Script-based environments
except NameError:
    base_dir = os.getcwd()  # Interactive environments (e.g., Jupyter, IPython)

# Build the path to the ini file
config_path = os.path.join(base_dir, '..', 'postgresql_initial_setup', 'db_config.ini')

# Normalize the path (resolve '..' to parent directory)
config_path = os.path.normpath(config_path)

# Load the configuration file
config = configparser.ConfigParser()
config.read(config_path)

# print(f"Configuration file path: {config_path}")

# Database connection details
DB_CONFIG = {
    'dbname': 'etl_proj_oms',
    'user': config['database']['user'],
    'password': config['database']['password'],
    'host': config['database']['host'],
    'port': config['database']['port']
}


START_DATE = datetime(2024, 1, 1)

# Connect to database
def connect_to_db(db_name):
    db_config = DB_CONFIG.copy()
    db_config["dbname"] = db_name
    return psycopg2.connect(**db_config)

# Populate data for etl_proj_oms
def populate_oms():
    conn = connect_to_db("etl_proj_oms")
    cursor = conn.cursor()

    orders = []
    order_items = []

    for i in range(1, 101):
        order_id = fake.uuid4()
        parcel_id = fake.uuid4()
        checkout_id = fake.uuid4()
        buyer_id = fake.uuid4()
        shop_id = fake.uuid4()
        payment_option_id = random.randint(1, 5)
        shipping_channel_id = random.randint(1, 5)
        delivery_detail_id = fake.uuid4()
        order_create_time = START_DATE + timedelta(days=random.randint(0, 365))
        ship_by_date = order_create_time + timedelta(days=random.randint(1, 7))
        shipping_fee = round(random.uniform(5, 50), 2)
        gmv = round(random.uniform(100, 1000), 2)
        wh_id = fake.uuid4()

        orders.append((order_id, parcel_id, checkout_id, buyer_id, shop_id, payment_option_id,
                       shipping_channel_id, delivery_detail_id, order_create_time, ship_by_date,
                       shipping_fee, gmv, wh_id))

        for _ in range(random.randint(1, 5)):
            sku_id = fake.uuid4()
            qty = random.randint(1, 10)
            gmv_item = round(gmv * qty * random.uniform(0.1, 0.3), 2)
            order_items.append((order_id, sku_id, shop_id, qty, gmv_item))

    # Create order table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS "order" (
            order_id UUID PRIMARY KEY,
            parcel_id UUID,
            checkout_id UUID,
            buyer_id UUID,
            shop_id UUID,
            payment_option_id INTEGER,
            shipping_channel_id INTEGER,
            delivery_detail_id UUID,
            order_create_time TIMESTAMP,
            ship_by_date TIMESTAMP,
            shipping_fee FLOAT,
            gmv FLOAT,
            wh_id UUID
        );
    """)

    # Create order item table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS order_item (
            order_id UUID,
            sku_id UUID,
            shop_id UUID,
            qty INTEGER,
            gmv FLOAT,
            PRIMARY KEY (order_id, sku_id)
        );
    """)

    # Insert data into order table
    execute_batch(cursor, """
        INSERT INTO "order" (order_id, parcel_id, checkout_id, buyer_id, shop_id, payment_option_id, 
        shipping_channel_id, delivery_detail_id, order_create_time, ship_by_date, 
        shipping_fee, gmv, wh_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, orders)

    # Insert data into order item table
    execute_batch(cursor, """
        INSERT INTO order_item (order_id, sku_id, shop_id, qty, gmv)
        VALUES (%s, %s, %s, %s, %s)
    """, order_items)

    conn.commit()
    cursor.close()
    conn.close()

# Function to generate new daily data for OMS
def generate_daily_data_oms():
    conn = connect_to_db("etl_proj_oms")
    cursor = conn.cursor()

    new_orders = []
    new_order_items = []
    today = datetime.now()

    for i in range(10):  # Generate 10 new orders daily
        order_id = fake.uuid4()
        parcel_id = fake.uuid4()
        checkout_id = fake.uuid4()
        buyer_id = fake.uuid4()
        shop_id = fake.uuid4()
        payment_option_id = random.randint(1, 5)
        shipping_channel_id = random.randint(1, 5)
        delivery_detail_id = fake.uuid4()
        order_create_time = today
        ship_by_date = today + timedelta(days=random.randint(1, 7))
        shipping_fee = round(random.uniform(5, 50), 2)
        gmv = round(random.uniform(100, 1000), 2)
        wh_id = fake.uuid4()

        new_orders.append((order_id, parcel_id, checkout_id, buyer_id, shop_id, payment_option_id,
                           shipping_channel_id, delivery_detail_id, order_create_time, ship_by_date,
                           shipping_fee, gmv, wh_id))

        for _ in range(random.randint(1, 3)):
            sku_id = fake.uuid4()
            qty = random.randint(1, 10)
            gmv_item = round(gmv * qty * random.uniform(0.1, 0.3), 2)
            new_order_items.append((order_id, sku_id, shop_id, qty, gmv_item))

    execute_batch(cursor, """
        INSERT INTO "order" (order_id, parcel_id, checkout_id, buyer_id, shop_id, payment_option_id, 
        shipping_channel_id, delivery_detail_id, order_create_time, ship_by_date, 
        shipping_fee, gmv, wh_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, new_orders)

    execute_batch(cursor, """
        INSERT INTO order_item (order_id, sku_id, shop_id, qty, gmv)
        VALUES (%s, %s, %s, %s, %s)
    """, new_order_items)

    conn.commit()
    cursor.close()
    conn.close()

if __name__ == "__main__":
    populate_oms()
    generate_daily_data_oms()


UndefinedColumn: column "order_create_time" of relation "order" does not exist
LINE 3:         shipping_channel_id, delivery_detail_id, order_creat...
                                                         ^
